# Binäre Klassifikation anhand eines Spam-Filters

## Thema
In dieser Binären Klassifikation handelt es sich um die Vorhersage auf das Eintreffen einer Spam-Email oder einer Ham-Email. Um dies herauszufinden wird ein Datensatz der mehrere Spam-/Ham-Emails mit EmailText enthält verwendet.

## Technische Vorbereitung
Damit alle notwendigen Funktionen und Befehle zur Verfügung stehen, müssen zunächst einige Module mit ihren jeweiligen Klassen importiert werden. Dazu zählt pandas und scikit-learn. Alle Module werden für spätere Datenanalysemethoden genutzt.


In [49]:
import pandas as pd
#from pandas import set_option
from matplotlib import pyplot
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

#from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier 

## Analyse des Datensatzes
Der Datensatz liegt hierbei in einer ".csv" - Datei und beinhaltet 2000 Datensätze. Er ist in zwei Spalten untergliedert. Die Spalte Label beinhaltet die Gruppen Spam und Ham die den jeweiligen Datensatz zugeordnet sind. Bei der zweiten Spalte handelt es sich um den EmailText.


In [51]:
dataframe = pd.read_csv("spam.csv")
print(dataframe.head())
print("")
print(dataframe.describe())


  Label                                          EmailText
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

       Label               EmailText
count   2000                    2000
unique     2                    1925
top      ham  Sorry, I'll call later
freq    1720                      13


Sowohl Label als auch der EmailText besitzen den Datentyp object.

In [52]:
print(dataframe.dtypes)


Label        object
EmailText    object
dtype: object


Durch den folgenden Code wurde die Anzahl von Spam- bzw. Ham-Emails in dem Datensatz gezählt. Dabei fällt auf, dass es sehr unausgeglichen ist und es sich bei den meisten Nachrichten um hamlose Emails handelt.

In [53]:
hamCount = dataframe.apply(lambda x: True if x["Label"] == "ham" else False, axis=1)
spamCount = dataframe.apply(lambda x: True if x["Label"] == "spam" else False, axis=1)

print("Anzahl ham  - Emails: ", len(hamCount[hamCount == True].index))
print("Anzahl spam - Emails: ", len(spamCount[spamCount == True].index))


Anzahl ham  - Emails:  1720
Anzahl spam - Emails:  280


## Traings- und Testdaten
Im nächsten Schritt werden die Daten in den Spalten separiert und danach gruppiert. Die Trainingsdaten werden für das Lernen der Muster und Zusammenhänge in den Daten verwendet. Der Algorithmus nutzt diese Daten um daraus zu lernen. Bei den Testdaten handelt es sich um Daten mit der gleichen Wahrscheinlichkeitsverteilung. Diese Daten werden vom Algorithmus vorher nicht genutzt. Mit ihnen kann nachgeweisen werden mit welcher Qualität der Algorithmus auf neue Daten reagieren wird. Die Verteilung verläuft auf 80% Trainingsdaten und 20% Testdaten.

In [54]:
x = dataframe["EmailText"]
y = dataframe["Label"]

x_train,y_train = x[0:1600],y[0:1600]
x_test,y_test = x[1600:],y[1600:]

## Transformieren der Daten
Nachdem die Daten gruppiert wurden müssen die Strings in numerische Werte umgewandelt werden, damit der Klassifikator die Daten nutzen kann. Scikit-Learn stellt hierbei einen "CountVectorizer" zur Verfügung. Er konvertiert eine Sammlung von Text Dokumenten in eine Matrix mit Token-Zahlen. Die Matrix baut sich aus 1600 Zeilen (EmailTexte) und 4398 Spalten (Anzahl unterschiedlicher Wörter) zusammen.

In [55]:
cv = CountVectorizer()  
featuresX = cv.fit_transform(x_train)

print(featuresX.shape)
print(featuresX.toarray())


(1600, 4398)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## Evaluierung der Algorithmen
Im folgenden Code wird mit Hilfe der Genauigkeit der beste Algorithmus herausgefunden. Die Genauigkeit wird dabei anhand des Mittelwertes und der Standardabweichung gemessen. Es stellt sich heraus, dass die logistic regression und der Support-Vector-Machine-Klassifikator am besten geeignet sind.

In [44]:
models = []
models.append(("LR", LogisticRegression()))
models.append(("NB", GaussianNB()))
models.append(("SVM", SVC()))

results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=10, shuffle=True)
    cv_results = cross_val_score(model, featuresX.toarray(), y_train, cv=kfold, scoring="accuracy")
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.970000 (0.015258)
NB: 0.918125 (0.022439)
SVM: 0.961250 (0.013636)


## Evaluierung der Algorithmen mit Hilfe der Standardisierung
In diesem Code werden alle Merkmale so transformiert, dass sie einen Mittelwert von 0 und eine Standardabweichung von 1 besitzen. Durch die Verwendung von pipelines wird Datenverlust vermieden. Dabei fällt auf das die Genauigkeit bei allen drei Algorithem gesunken ist. Die logistic regression und der Support-Vektor-Machine-Klassifikator sind trotzdem noch die bessern.

In [45]:
pipelines = []
pipelines.append(("ScaledLR", Pipeline([("Scaler", StandardScaler()), ("LR", LogisticRegression())])))
pipelines.append(("ScaledNB", Pipeline([("Scaler", StandardScaler()), ("NB", GaussianNB())])))
pipelines.append(("ScaledSVM", Pipeline([("Scaler", StandardScaler()), ("SVM", SVC())])))

results =[]
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=10, shuffle=True)
    cv_results = cross_val_score(model, featuresX.toarray(), y_train, cv=kfold, scoring="accuracy")
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)


ScaledLR: 0.955625 (0.020243)
ScaledNB: 0.905000 (0.024012)


KeyboardInterrupt: 

## Verbesserung der Algorithmen
Durch das Verändern/Anpassen von Parametern für einen jeweiligen Algroithmus kann seine Genauigkeit erhöht werde.n

### Verbesserung des Support-Vector-Machine-Klassifikator
Beim SVM lassen sich zwei Parameter anpassen. Zum einen der Kernel und zum anderen der C-Wert. Der Kernel versucht den Trainingsvektor in verschiedene höherdimensionale Räume zu überführen, damit die Vektoren sich leichter trennen lassen. Beim C-Wert wird dem Klassifikator mitgeteilt wie sehr er eine Fehlklassifizierung vermeiden soll. Hierbei entsteht eine Genauigkeit von 97,85% mit dem C-Wert von 0,1 und einem Linearen kernel.

In [11]:
c_values = [0.1, 0.5, 1.0, 1.5, 2.0]
kernel_values = ["linear", "poly", "rbf", "sigmoid"]
param_grid = dict(C=c_values, kernel=kernel_values)
model = SVC()
kfold = KFold(n_splits=10, shuffle=True)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring="accuracy", cv=kfold)
grid_result = grid.fit(featuresX.toarray(), y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


Best: 0.978500 using {'C': 0.1, 'kernel': 'linear'}


### Verbesserung der logistic regression
Die logistic regression lässt sich durch zwei Parameter verändern. Zum einen den "solver" und zum anderen die "penalty" in Abhängigkeit der C-Werte. 
In der logistic regression gibt es fünf verschiedene "solver". Jeder einzelne versucht die Gewichtung der Parameterso anzupassen das die Kosten am niedrigsten sind. Die newton Methode nutzt die Hesse-Matrix, sie berechnet die zweite Ableitung. lbfgs steht für Limited-memory Broyden-Fletcher-Goldfarb-Shanno und es nähert sich der Aktualisierung der zweiten Ableitung der Matrix mit Gradientenbewertung. liblinear bedeutet Library for Large Linear Classification. Es verwendet den Koordinatenabstiegsalgorithmus und bewegt sich jeweils in eine Richtung zum Minimum.
Mit Hilfe der penalty wird die Regualarisierung durchgeführt. Durch den C-Wert wird die Stärke der penalty kontrolliert.
Es entsteht eine Genauigkeit von 97,75% mit dem C-Wert von 100, der penalty von l2 und der liblinearen Methoden.


In [24]:
solvers = ["newton-cg", "lbfgs", "liblinear"]
penalty = ["l2"]
c_values = [0.01, 0.1, 1.0, 10, 100]
param_grid = dict(solver=solvers, penalty=penalty, C=c_values)
model = LogisticRegression()
kfold = KFold(n_splits=10, shuffle=True)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring="accuracy", cv=kfold)
grid_result = grid.fit(featuresX.toarray(), y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


Best: 0.977500 using {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}


## Random Forest

In [27]:
model = RandomForestClassifier()
kfold = KFold(n_splits=10, shuffle=True)
cv_results = cross_val_score(model, featuresX.toarray(), y_train, cv=kfold, scoring="accuracy")
msg = "RF: %f (%f)" % (cv_results.mean(), cv_results.std())
print(msg)  

RF: 0.967500 (0.011673)


## Finaler Test
Die beste Genauigkeit erzielte der Support-Vektor-Machine-Klassifikator mit den Parametern: 
    C-Wert: 0,1
    kernel: linear.
Im folgenden Code wird der Klassifikator mit den Testdaten überprüft, um eine Genauigkeit der Vorhersage neuer Werte zu treffen.

In [56]:
model = SVC(C=0.1, kernel="linear")
model.fit(featuresX.toarray(), y_train)
validateX = cv.fit_transform(x_test)
predictions = model.predict(validateX.toarray())
print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_train, predictions))
print(classification_report(y_train, predictions))



ValueError: X.shape[1] = 1812 should be equal to 4398, the number of features at training time

## Quellen
https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

https://towardsdatascience.com/dont-sweat-the-solver-stuff-aea7cddc3451

Brownlee, Jason (2018): Machine Learning Mastery with Python, S. 144 - 161 